In [19]:
import pdfplumber

with pdfplumber.open("RelatorioFechamentoCaixaBobina.pdf") as pdf:
    page = pdf.pages[0]
    texto = page.extract_text()

In [ ]:
import re

def parse_dav(texto):
    padrao = r"[A-Z ]+\s+([A-Z]{2})\s+(\d+,\d+)"
    return re.findall(padrao, texto)

def parse_sangrias(texto):
    padrao = r"DAV\s+([A-Z ]+)\s+(\d+,\d+)"
    return re.findall(padrao, texto)

def separar_secoes(texto):
    # Marcações claras no relatório
    secoes = {
        "dav": "",
        "sangrias": "",
        "totalizadores": ""
    }

    # Quebra em blocos
    linhas = texto.splitlines()

    secao_atual = None

    for linha in linhas:
        l = linha.strip().upper()

        if "DAV" in l and "VENDAS" in l:
            secao_atual = "dav"
            continue

        if "LISTA DAS SANGRIAS" in l:
            secao_atual = "sangrias"
            continue

        if "TOTALIZADORES" in l:
            secao_atual = "totalizadores"
            continue

        if secao_atual:
            secoes[secao_atual] += linha + "\n"

    return secoes

def parse_relatorio(texto):
    secoes = separar_secoes(texto)

    # DAV (primeiro bloco)
    dav = parse_dav(secoes["dav"])

    # Sangrias
    sangrias = parse_sangrias(secoes["sangrias"])

    # Totalizadores (parte final)
    totalizadores = parse_dav(secoes["totalizadores"])

    return {
        "dav": dav,
        "sangrias": sangrias,
        "totalizadores": totalizadores
    }


{'dav': [('DN', '183,00'), ('CD', '182,00'), ('PX', '328,00')], 'sangrias': [('OVOS', '24,00'), ('VASSOURA', '30,00')], 'totalizadores': [('DN', '129,00'), ('CD', '182,00'), ('PX', '328,00')]}


In [ ]:
import openpyxl
from openpyxl.styles import PatternFill
relatorio = openpyxl.load_workbook("CAIXA_format.xlsx", data_only=False)
caixa = relatorio['Planilha1']

dav = parse_relatorio(texto)['dav']
mapa = {
    "DN": "DINHEIRO",
    "CD": "DEBITO",
    "PX": "PIX",
    "PZ": "PRAZO"
}

sangria = parse_relatorio(texto)['sangrias']
    

[('DN', '183,00'), ('CD', '182,00'), ('PX', '328,00')]
